In [26]:
import pandas as pd
import datetime as dt
from game_log import load_current_line
import matplotlib.pyplot as plt

today = dt.datetime.today()
year = today.year
month = today.month
day = today.day
today_str = f'{year}_{month}_{day}'

pp_path = f'Lines/pp/pp_{today_str}.csv'
unabated_path = f'Lines/unabated/unabated_{today_str}.csv'
dog_path = f'Lines/dog/doglines_{today.month}_{today.day}.csv'

pp_lines = load_current_line(pp_path,'prop_id','time')
unabated = load_current_line(unabated_path,'prop_id','time')
dog_lines = pd.read_csv(dog_path,index_col=0)

In [27]:
#pp_lines['date'] = pp_lines["date"].apply(lambda x: pd.to_datetime(x,utc=True))
#unabated["event_time"].apply(lambda x: pd.to_datetime(x,utc=True))

In [28]:
nba = pp_lines.loc[pp_lines.league_id==7]
combined = unabated.merge(pp_lines[['player','line','stat']],how='inner', on=['player','stat'],suffixes=['_u','_pp'])
combined

,prop_id,player,player_id,line_u,stat,league_id,event_time,opp,team,count,over_prob,under_prob,pred,time,line_pp
0,a71b0dd6-7,Cole Kmet,38204,34.5,Receiving Yards,1,2023-11-09 19:15:00-06:00,CAR,CHI,3,0.515621,0.484379,37.178531,2023-11-09 15:28:00,37.5
1,a71b0dd6-7,Cole Kmet,38204,35.5,Receiving Yards,1,2023-11-09 19:15:00-06:00,CAR,CHI,3,0.503372,0.496628,37.178531,2023-11-09 15:28:00,37.5
2,a71b0dd6-7,Cole Kmet,38204,37.5,Receiving Yards,1,2023-11-09 19:15:00-06:00,CAR,CHI,8,0.496834,0.503166,37.178531,2023-11-09 15:28:00,37.5
3,a71b0dd6-7,Cole Kmet,38204,39.5,Receiving Yards,1,2023-11-09 19:15:00-06:00,CAR,CHI,3,0.500000,0.500000,37.178531,2023-11-09 15:28:00,37.5
4,a7788945-6,Tyjae Spears,410108,20.5,Rush Yards,1,2023-11-12 12:00:00-06:00,TB,TEN,12,0.505699,0.494301,20.732363,2023-11-09 15:28:00,20.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,57bc865f-2,Myles Turner,42819,25.5,PTS+TRB+AST,3,2023-11-09 18:00:00-06:00,MIL,IND,7,0.513793,0.486207,25.844352,2023-11-09 15:28:00,25.5
969,57cd22ae-a,Wan'Dale Robinson,33948,30.5,Receiving Yards,1,2023-11-12 15:25:00-06:00,DAL,NYG,1,0.510121,0.489879,31.619456,2023-11-09 15:28:00,31.5
970,57cd22ae-a,Wan'Dale Robinson,33948,31.5,Receiving Yards,1,2023-11-12 15:25:00-06:00,DAL,NYG,7,0.503809,0.496191,31.619456,2023-11-09 15:28:00,31.5
971,556b11cd-a,Antonio Gibson,36736,15.5,Receiving Yards,1,2023-11-12 15:25:00-06:00,SEA,WAS,1,0.520305,0.479695,16.659262,2023-11-09 15:28:00,16.5


In [29]:
combined = combined[['prop_id', 'player','line_u','line_pp', 'player_id', 'stat', 
       'event_time', 'opp', 'team', 'count', 'over_prob', 'under_prob', 'pred',]]

In [30]:
combined.loc[combined.line_u == combined.line_pp].sort_values(by='over_prob').head(15)

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred
423,bc62eda3-6,Alex Anzalone,0.25,0.25,34194,Sacks,2023-11-12 15:05:00-06:00,LAC,DET,1,0.304487,0.695513,0.361864
291,eda3c7bf-3,Chuba Hubbard,2.50,2.50,35012,Receptions,2023-11-09 19:15:00-06:00,CHI,CAR,15,0.468665,0.531335,2.546704
389,c46c1572-5,Myles Turner,0.50,0.50,42819,STL,2023-11-09 18:00:00-06:00,MIL,IND,11,0.469479,0.530521,0.632540
181,8294bf5c-7,Goga Bitadze,17.50,17.50,42809,PTS+TRB+AST,2023-11-09 20:30:00-06:00,ATL,ORL,7,0.477941,0.522059,17.446003
702,1d1a65f2-8,Jalen Suggs,15.50,15.50,43426,PTS+TRB,2023-11-09 20:30:00-06:00,ATL,ORL,8,0.478873,0.521127,15.467618
427,c0c535f1-e,Alvin Kamara,51.50,51.50,38137,Rush Yards,2023-11-12 12:00:00-06:00,MIN,NO,10,0.479506,0.520494,50.944313
518,d551a0a6-4,D'Onta Foreman,15.50,15.50,36680,Rush Attempts,2023-11-09 19:15:00-06:00,CAR,CHI,1,0.479695,0.520305,15.169016
422,bcee2dd8-0,Jalen Johnson,12.50,12.50,43456,PTS,2023-11-09 20:30:00-06:00,ORL,ATL,9,0.480544,0.519456,12.500000
832,709eb8c5-8,Jauan Jennings,17.50,17.50,35057,Receiving Yards,2023-11-12 12:00:00-06:00,JAX,SF,10,0.482042,0.517958,17.332233
919,4507e269-7,Aaron Nesmith,10.50,10.50,242371,PTS,2023-11-09 18:00:00-06:00,MIL,IND,9,0.483366,0.516634,10.531793


In [31]:
combined.loc[combined.line_u == combined.line_pp].sort_values(by='over_prob').tail(15)

,prop_id,player,line_u,line_pp,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred
194,817ff1e5-f,Bogdan Bogdanovic,14.5,14.5,42909,PTS+TRB,2023-11-09 20:30:00-06:00,ORL,ATL,6,0.517321,0.482679,15.000219
48,a08523e6-7,Jalen Johnson,22.5,22.5,43456,PTS+TRB+AST,2023-11-09 20:30:00-06:00,ORL,ATL,8,0.517560,0.482440,22.939108
69,b7813e60-a,Clint Capela,9.5,9.5,42911,PTS,2023-11-09 20:30:00-06:00,ORL,ATL,13,0.518764,0.481236,9.837892
143,afb59f43-5,Adam Thielen,6.5,6.5,36815,Receptions,2023-11-09 19:15:00-06:00,CHI,CAR,16,0.519308,0.480692,6.793091
431,c2c9b32d-8,Miles Sanders,10.5,10.5,34624,Receiving Yards,2023-11-09 19:15:00-06:00,CHI,CAR,7,0.519433,0.480567,11.334403
820,4023dc17-9,Tyrese Haliburton,3.5,3.5,42813,TRB,2023-11-09 18:00:00-06:00,MIL,IND,13,0.522027,0.477973,3.775901
265,e94cdd8b-7,DeAndre Hopkins,4.5,4.5,242390,Receptions,2023-11-12 12:00:00-06:00,TB,TEN,11,0.523582,0.476418,4.797294
607,3f4e2512-b,Onyeka Okongwu,8.5,8.5,43444,PTS,2023-11-09 20:30:00-06:00,ORL,ATL,9,0.524297,0.475703,8.847333
509,cd2aa628-2,Bogdan Bogdanovic,14.5,14.5,42909,PTS+AST,2023-11-09 20:30:00-06:00,ORL,ATL,6,0.524797,0.475203,15.042574
424,bb7afc48-0,Paolo Banchero,4.5,4.5,42838,AST,2023-11-09 20:30:00-06:00,ATL,ORL,14,0.524958,0.475042,4.804813


In [32]:
fantasy = dog_lines.merge(pp_lines[~pp_lines.league_id.isin([192,84])],how='inner',on=['player','stat'],suffixes=['_dog','_pp'])
fantasy['diff'] = fantasy['line_dog'] - fantasy['line_pp']
fantasy['Z'] = fantasy['diff']/fantasy['line_pp']
fantasy = fantasy[fantasy['diff'] != 0]
fantasy = fantasy[['player','stat','diff','Z','line_dog','line_pp','league_name']].sort_values(by='Z').head(32)
fantasy = fantasy.loc[fantasy.league_name != 'NFL SPRINTS']
fantasy

,player,stat,diff,Z,line_dog,line_pp,league_name


In [33]:
pd.set_option('display.max_rows', 33)

In [34]:
combined.merge(fantasy,on=['player','stat'])

,prop_id,player,line_u,line_pp_x,player_id,stat,event_time,opp,team,count,over_prob,under_prob,pred,diff,Z,line_dog,line_pp_y,league_name
